In [ ]:
!pip install wget
!pip install awscli
import os
os._exit(00)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.9 MB 13.4 MB/s 
     |████████████████████████████████| 9.1 MB 70.2 MB/s 
     |████████████████████████████████| 636 kB 74.2 MB/s 
     |████████████████████████████████| 548 kB 51.0 MB/s 
     |████████████████████████████████| 79 kB 8.1 MB/s 
     |████████████████████████████████| 140 kB 68.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9
    Uninstalling rsa-4.9:
      Successfully uninstalled rsa-4.9
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
  Attempting unin

In [2]:
import os
!export AWS_SHARED_CREDENTIALS_FILE=/content/drive/MyDrive/config/awscli.ini
path="/content/drive/MyDrive/config/awscli.ini"
os.environ['AWS_SHARED_CREDENTIALS_FILE']=path
import awscli

In [5]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab import auth
auth.authenticate_user()

import pandas as pd
import wget
import numpy as np
import json
import os
import pickle
import yaml
import shutil
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras

import warnings; warnings.simplefilter('ignore')

from drive.MyDrive.mushroom_classification.new import src

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
CFG = yaml.safe_load(open('drive/MyDrive/mushroom_classification/new/src/config.YAML', 'rb'))

In [ ]:
train, val, test = src.get_images()
train = pd.read_csv(CFG['path'] + 'dataframes/train.csv')
val = pd.read_csv(CFG['path'] + 'dataframes/val.csv')
train = pd.concat([train], ignore_index=True)
val = pd.concat([val], ignore_index=True)
train['species_per_genus'] = train.groupby('category_id')['category_id'].transform('count')
val['species_per_genus'] = val.groupby('category_id')['category_id'].transform('count')

In [ ]:
images = !ls images/
new_images = images
image_map = []
for image in new_images:
    new_image = ' '.join(image.split('_')[1:])
    id = train[train['name'] == new_image]['category_id'].unique()
    image_map.append(str(id))

!mkdir val

for dir in images:
    newdir = '/'.join(['val', dir[1:]])
    os.mkdir(newdir)

val['filepath'] = 'images/' + val['sub_dir'] + '/' + val['image_name']
val['new_filepath'] = 'val/' + val['sub_dir'] + '/' + val['image_name']

missing_list = []
for idx, row in val[['filepath', 'new_filepath']].iterrows():
    try:
        os.rename(row['filepath'], row['new_filepath'])
    except FileNotFoundError:
        missing_list.append(row['new_filepath'])

['[0]', '[1]', '[2]', '[3]', '[4]', '[5]', '[6]', '[7]', '[8]', '[9]', '[10]', '[11]', '[12]', '[13]', '[14]', '[15]', '[16]', '[17]', '[18]', '[19]', '[20]', '[21]', '[22]', '[23]', '[24]', '[25]', '[26]', '[27]', '[28]', '[29]', '[30]', '[31]', '[32]', '[33]', '[34]', '[35]', '[]', '[37]', '[38]', '[]', '[40]', '[]', '[42]', '[43]', '[]', '[45]', '[46]', '[47]', '[48]', '[49]', '[50]', '[51]', '[52]', '[53]', '[54]', '[55]', '[56]', '[57]', '[58]', '[59]', '[60]', '[61]', '[62]', '[63]', '[64]', '[65]', '[66]', '[67]', '[68]', '[69]', '[70]', '[71]', '[72]', '[73]', '[74]', '[75]', '[76]', '[77]', '[78]', '[79]', '[80]', '[81]', '[82]', '[83]', '[84]', '[85]', '[86]', '[87]', '[88]', '[89]', '[90]', '[91]', '[92]', '[93]', '[94]', '[95]', '[96]', '[97]', '[98]', '[99]', '[100]', '[101]', '[102]', '[103]', '[104]', '[105]', '[106]', '[107]', '[108]', '[109]', '[110]', '[111]', '[112]', '[113]', '[114]', '[115]', '[116]', '[117]', '[118]', '[119]', '[120]', '[121]', '[122]', '[123]', '

In [ ]:
train_ds = keras.preprocessing.image_dataset_from_directory('images/', labels='inferred', label_mode='categorical', batch_size=124, image_size=(150, 150))
val_ds = keras.preprocessing.image_dataset_from_directory('val/', labels='inferred', label_mode='categorical', batch_size=124, image_size=(150, 150))

Found 85581 files belonging to 1394 classes.
Found 4179 files belonging to 1395 classes.


# AWS

In [3]:
# import tarfile

# !aws s3 cp s3://ml-inat-competition-datasets/2021/train_mini.tar.gz .
# !aws s3 cp s3://ml-inat-competition-datasets/2021/val.tar.gz .
# !aws s3 cp s3://ml-inat-competition-datasets/2021/val.json.tar.gz .
# !aws s3 cp s3://ml-inat-competition-datasets/2021/train_mini.json.tar.gz .

# with tarfile.open('train_mini.tar.gz') as f:
#     f.extractall()
# with tarfile.open('val.tar.gz') as f:
#     f.extractall()
# with tarfile.open('val.json.tar.gz') as f:
#     f.extractall()
# with tarfile.open('train_mini.json.tar.gz') as f:
#     f.extractall()

val = src.read_aws_jsons('val.json')
train_mini = src.read_aws_jsons('train_mini.json')

def select_only_fungi(name, df):
    df_list = df['sub_dir'].values.tolist()
    dirs = os.listdir(name)
    remove_list = [x for x in dirs if x not in df_list]

    for path in remove_list:
        shutil.rmtree(name+'/'+path)

    for path in os.listdir(name):
        split = path.split('_')
        num = split[0]
        newname = split[-2:]
        newname.insert(0, num)
        new = '_'.join(newname)
        os.rename(name+'/'+path, name+'/'+new)

select_only_fungi('val', val)
select_only_fungi('train_mini', train_mini)

train_dirs = !ls train_mini
tm_trunc_dirs = ["_".join(x.split('_')[1:]) for x in train_dirs]

download: s3://ml-inat-competition-datasets/2021/train_mini.tar.gz to ./train_mini.tar.gz
download: s3://ml-inat-competition-datasets/2021/val.tar.gz to ./val.tar.gz
download: s3://ml-inat-competition-datasets/2021/val.json.tar.gz to ./val.json.tar.gz
download: s3://ml-inat-competition-datasets/2021/train_mini.json.tar.gz to ./train_mini.json.tar.gz


In [ ]:
# for path in os.listdir('images'):
#     split = path.split('_')
#     name = split[1:3]
#     new = '_'.join(name)
#     tm_list = []
#     print(new)

#     #os.rename('train_mini/'+path, 'train_mini/'+new)
#     break

Marasmius_bulliardii


In [ ]:
cat_names = val['name'].unique().tolist()
categories = pd.DataFrame(cat_names)
categories.reset_index(inplace=True)
categories.columns = ['label', 'name']
categories.set_index('name', inplace=True)
val.set_index('name', inplace=True)
train_mini.set_index('name', inplace=True)
val = val.merge(categories, on='name', how='left')
train_mini = train_mini.merge(categories, on='name', how='left')

In [ ]:
train_mini.reset_index(inplace=True)
val.reset_index(inplace=True)
train_mini['sub_dir'] = train_mini.apply(lambda x: "_".join((x['sub_dir'].split('_')[0], x['genus'], x['specific_epithet'])), axis=1)
val['sub_dir'] = val.apply(lambda x: "_".join((x['sub_dir'].split('_')[0], x['genus'], x['specific_epithet'])), axis=1)

In [ ]:
val['filepath'] = 'val/'+val['sub_dir']+'/'+val['image_name']
train_mini['filepath'] = 'train_mini/'+train_mini['sub_dir']+'/'+train_mini['image_name']

In [ ]:
val

,name,width,height,date,latitude,longitude,category_id,genus,specific_epithet,image_name,sub_dir,day,filepath,label
0,Pholiota squarrosa,500,380,2011-09-14,59.32726,18.13697,5585,Pholiota,squarrosa,4a1df4d1-4cd8-469d-8496-7b025a5b73cd.jpg,05585_Pholiota_squarrosa,257,content/val/05585_Pholiota_squarrosa/4a1df4d1-...,0
1,Craterellus tubaeformis,415,500,2011-09-09,59.18437,18.26948,5626,Craterellus,tubaeformis,f076d7a7-7886-4448-afef-cc15dd9e027b.jpg,05626_Craterellus_tubaeformis,252,content/val/05626_Craterellus_tubaeformis/f076...,1
2,Calocera viscosa,368,500,2011-09-13,50.79915,-1.59319,5718,Calocera,viscosa,b377dcec-8bff-4869-80c9-d0d25a02ab13.jpg,05718_Calocera_viscosa,256,content/val/05718_Calocera_viscosa/b377dcec-8b...,2
3,Gymnopilus junonius,480,360,2012-10-08,42.52333,-71.61266,5525,Gymnopilus,junonius,cb458ce5-4040-414d-82ed-261a2b3ed538.jpg,05525_Gymnopilus_junonius,282,content/val/05525_Gymnopilus_junonius/cb458ce5...,3
4,Hydnellum peckii,500,375,2012-10-09,45.90783,-121.70475,5716,Hydnellum,peckii,a7371564-d09e-40ee-ad12-2a2b869d45b4.jpg,05716_Hydnellum_peckii,283,content/val/05716_Hydnellum_peckii/a7371564-d0...,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3405,Fomitopsis mounceae,375,500,2019-08-15,60.78755,-135.09470,5653,Fomitopsis,mounceae,9207cf69-dd3a-4a36-a4f3-c7d82fa7ac3b.jpg,05653_Fomitopsis_mounceae,227,content/val/05653_Fomitopsis_mounceae/9207cf69...,144
3406,Bondarzewia berkeleyi,375,500,2019-07-02,39.02756,-77.27073,5699,Bondarzewia,berkeleyi,51cb29ad-1a72-42e8-af42-07ec73579219.jpg,05699_Bondarzewia_berkeleyi,183,content/val/05699_Bondarzewia_berkeleyi/51cb29...,270
3407,Amanita fulva,375,500,2017-09-19,39.11140,-77.85755,5491,Amanita,fulva,b7b4ea89-5189-4eb2-b5c9-dcce9eaed00a.jpg,05491_Amanita_fulva,262,content/val/05491_Amanita_fulva/b7b4ea89-5189-...,176
3408,Hygrocybe conica,375,500,2019-09-15,56.77604,-5.20438,5519,Hygrocybe,conica,75cfadfd-d61c-4316-92b8-b47b3e0b711e.jpg,05519_Hygrocybe_conica,258,content/val/05519_Hygrocybe_conica/75cfadfd-d6...,90


# Creating Datasets

In [ ]:
import PIL
import PIL.Image
import tensorflow as tf
AUTOTUNE = tf.data.AUTOTUNE

t_paths = train_mini['filepath']
v_paths = val['filepath']
t_class = train_mini['label']
v_class = val['label']

def load_img(filepath):
    img = tf.io.read_file(filepath)
    img = tf.io.decode_jpeg(img, 3)
    img = tf.cast(img, tf.float32)
    img = applications.efficientnet_v2.preprocess_input(img)
    img = tf.image.resize(img, size=(150, 150))
    return img


train_ds = tf.data.Dataset.from_tensor_slices((t_paths, t_class)).map(
    lambda filepath, label: (load_img(filepath), label), 
    num_parallel_calls=AUTOTUNE).\
    cache().\
    shuffle(20000).\
    batch(CFG['BATCH_SIZE']).\
    prefetch(buffer_size=AUTOTUNE)
    
val_ds = tf.data.Dataset.from_tensor_slices((v_paths, v_class)).map(
    lambda filepath, label: (load_img(filepath), label), 
    num_parallel_calls=AUTOTUNE).\
    cache().\
    shuffle(4000).\
    batch(CFG['BATCH_SIZE']).\
    prefetch(buffer_size=AUTOTUNE)

In [ ]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

255.0

In [ ]:
%load_ext tensorboard
import datetime
from tensorflow.keras import applications 
from tensorflow.keras import layers
image_size = [150, 150]
!rm -rf ./logs/
AUTOTUNE = tf.data.experimental.AUTOTUNE

def lr_scheduler(epoch):
    return CFG['INITIAL_LR_RATE'] * tf.math.pow(CFG['LR_DECAY_FACTOR'], epoch//CFG['EPOCHS_PER_DECAY'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
optimizer = tf.keras.optimizers.Nadam(CFG['INITIAL_LR_RATE'])

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='loss', patience=CFG['ES_PATIENCE']),

    tf.keras.callbacks.LearningRateScheduler(
        lr_scheduler, verbose=1),

    tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
]

def create_model():
    input = layers.Input(shape=CFG['IMAGE_SIZE'] + [3], dtype=tf.float32)
    base = applications.efficientnet_v2.EfficientNetV2B0(include_top=False, input_tensor=input)
    avg = layers.GlobalAveragePooling2D()(base.output)
    dropout = layers.Dropout(CFG['DROPOUT_PERCENT'])(avg)
    x = layers.Dense(341, activation='softmax')(dropout)
    model = keras.Model(inputs=base.input, outputs=x)
    return model

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
num_train_examples = train_mini.shape[0]
num_val_examples = val.shape[0]
STEPS_PER_EPOCH = np.ceil(num_train_examples/CFG["BATCH_SIZE"])
VAL_STEPS = np.ceil(num_val_examples/CFG["BATCH_SIZE"])
model = create_model()
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top3 accuracy')])

In [ ]:
%time
history = model.fit(train_ds,
                    validation_data=val_ds,
                    validation_steps=VAL_STEPS,
                    epochs=5,
                    steps_per_epoch=STEPS_PER_EPOCH,
                    callbacks=callbacks)

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 9.54 µs

Epoch 1: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 1/5
267/267 [==============================] - 2328s 8s/step - loss: 5.1588 - accuracy: 0.0300 - top3 accuracy: 0.0103 - val_loss: 4.8714 - val_accuracy: 0.0411 - val_top3 accuracy: 0.0000e+00 - lr: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 2/5
267/267 [==============================] - 1998s 7s/step - loss: 4.2195 - accuracy: 0.0980 - top3 accuracy: 0.0090 - val_loss: 4.9334 - val_accuracy: 0.0554 - val_top3 accuracy: 0.0012 - lr: 0.0100

Epoch 3: LearningRateScheduler setting learning rate to 0.009700000286102295.
Epoch 3/5
220/267 [=======================>......] - ETA: 5:36 - loss: 3.7017 - accuracy: 0.1636 - top3 accuracy: 0.0079

KeyboardInterrupt: ignored

In [ ]:
len(train_mini['label'].unique())

341